In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
# Load AWS Credentials
spark.conf.set("fs.s3a.access.key", "")
spark.conf.set("fs.s3a.secret.key", "")

In [0]:
# Read HVFHV DataFrames and drop duplicate rows
HVFHV_df = spark.read.option("recursiveFileLookup", "true").parquet("s3a://capstone-techcatalyst-conformed/group4/hvfhv_data/*",multiLine=True, encoding="utf8")

In [0]:
# Calculate Trip Distance % of Total Trip Distance for the day.
HVFHV_df = HVFHV_df.withColumn('trip_distance_%', col('trip_miles')/ sum('trip_miles').over(Window.partitionBy(to_date("Pickup_datetime")))*100)

# Calculate Trip Duration % of Total Time Spent Active for the day.
HVFHV_df = HVFHV_df.withColumn('trip_duration_%', col('trip_time')/ sum('trip_time').over(Window.partitionBy(to_date("Pickup_datetime")))*100)

In [0]:
HVFHV_df = HVFHV_df.withColumn("year_", col("year"))
HVFHV_df = HVFHV_df.withColumn("month_", col("month"))
HVFHV_df = HVFHV_df.withColumn("day_", col("day_of_month"))

HVFHV_parquet_path = f"s3a://capstone-techcatalyst-transformed/group4/hvfhv_data/"

HVFHV_df.write.partitionBy(["year_", "month_", "day_"]).parquet(HVFHV_parquet_path)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2454545296654845>:7
      3 HVFHV_df = HVFHV_df.withColumn("day_", col("day_of_month"))
      5 HVFHV_parquet_path = f"s3a://capstone-techcatalyst-transformed/group4/hvfhv_data/"
----> 7 HVFHV_df.write.partitionBy(["year_", "month_", "day_"]).parquet(HVFHV_parquet_path)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._